from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

In [26]:
import torch
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example tokenization
inputs = tokenizer(['Hello world', 'Hi how are you'], padding=True, truncation=True, return_tensors='pt')
outputs = model(**inputs)

# Load emotions dataset
emotions = load_dataset('SetFit/emotion')

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# Tokenize and encode emotions dataset
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded.set_format('pt', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# Batch size
BATCH_SIZE = 64

# PyTorch DataLoader for train set
train_dataset = TensorDataset(
    emotions_encoded['train']['input_ids'],
    emotions_encoded['train']['attention_mask'],
    emotions_encoded['train']['token_type_ids'],
    emotions_encoded['train']['label']
)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# PyTorch DataLoader for test set
test_dataset = TensorDataset(
    emotions_encoded['test']['input_ids'],
    emotions_encoded['test']['attention_mask'],
    emotions_encoded['test']['token_type_ids'],
    emotions_encoded['test']['label']
)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Example batch from train_dataloader
batch = next(iter(train_dataloader))
input_ids, attention_mask, token_type_ids, labels = batch
print(input_ids, '\n\n', labels)

# BERT for classification model in PyTorch
class BERTForClassification(torch.nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTForClassification, self).__init__()
        self.bert = bert_model
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Instantiate the model
num_classes = 6
classifier = BERTForClassification(model, num_classes)

# Set optimizer and loss function
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    classifier.train()
    for batch in train_dataloader:
        input_ids, attention_mask, token_type_ids, labels = batch
        optimizer.zero_grad()
        logits = classifier(input_ids, attention_mask, token_type_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

# Evaluate on the test set
classifier.eval()
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, token_type_ids, labels = batch
        logits = classifier(input_ids, attention_mask, token_type_ids)
        predictions = torch.argmax(logits, dim=1)
        accuracy = torch.sum(predictions == labels).item() / len(labels)

print("Test Accuracy:", accuracy)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/aayushsonkar/Desktop/Grind/2023-2024/Me/Syncwise/ML/main/syncwise/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wn/qnn9lpvn7ds_v_jkgk6qs1400000gn/T/ipykernel_76201/3995972475.py", line 20, in <module>
    emotions = load_dataset('SetFit/emotion')
  File "/Users/aayushsonkar/Desktop/Grind/2023-2024/Me/Syncwise/ML/main/syncwise/venv/lib/python3.10/site-packages/datasets/load.py", line 2523, in load_dataset
    builder_instance = load_dataset_builder(
  File "/Users/aayushsonkar/Desktop/Grind/2023-2024/Me/Syncwise/ML/main/syncwise/venv/lib/python3.10/site-packages/datasets/load.py", line 2195, in load_dataset_builder
    dataset_module = dataset_module_factory(
  File "/Users/aayushsonkar/Desktop/Grind/2023-2024/Me/Syncwise/ML/main/syncwise/venv/lib/python3.10/site-packages/datasets/load.py", line 1846, in dataset_module